In [1]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

In [2]:
# Clear any logs from previous runs
!rm -rf ./logs/

In [3]:
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras
tf.debugging.experimental.enable_dump_debug_info('./logs/',
                                                 tensor_debug_mode="FULL_HEALTH", 
                                                 circular_buffer_size=-1)
from keras import backend as K
import numpy as np

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

2024-06-04 20:56:36.558911: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-04 20:56:36.558967: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-04 20:56:36.610119: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-04 20:56:36.740765: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-04 20:56:37.915095: W tensorflow/compiler/tf2

INFO:tensorflow:Enabled dumping callback in thread MainThread (dump root: ./logs/, tensor debug mode: FULL_HEALTH)
TensorFlow version:  2.15.1


In [4]:
data_size = 1000
# 80% of the data is for training.
train_pct = 0.8

train_size = int(data_size * train_pct)

# Create some input data between -1 and 1 and randomize it.
x = np.linspace(-1, 1, data_size)
np.random.shuffle(x)

# Generate the output data.
# y = 0.5x + 2 + noise
y = 0.5 * x + 2 + np.random.normal(0, 0.05, (data_size, ))

# Split into test and train pairs.
x_train, y_train = x[:train_size], y[:train_size]
x_test, y_test = x[train_size:], y[train_size:]

In [5]:
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model = keras.models.Sequential([
    keras.layers.Dense(16, input_dim=1),
    keras.layers.Dense(1),
])

model.compile(
    loss='mse', # keras.losses.mean_squared_error
    optimizer=keras.optimizers.SGD(learning_rate=0.2),
)

print("Training ... With default parameters, this takes less than 10 seconds.")
training_history = model.fit(
    x_train, # input
    y_train, # output
    batch_size=train_size,
    verbose=1, # Suppress chatty output; use Tensorboard instead
    epochs=20,
    validation_data=(x_test, y_test),
    callbacks=[tensorboard_callback],
)

print("Average test loss: ", np.average(training_history.history['loss']))

2024-06-04 20:56:43.476305: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Training ... With default parameters, this takes less than 10 seconds.
Epoch 1/20
1/1 [==============================] - 1s 815ms/step - loss: 4.0232 - val_loss: 0.0409
Epoch 2/20
1/1 [==============================] - 0s 37ms/step - loss: 0.0452 - val_loss: 0.0144
Epoch 3/20
1/1 [==============================] - 0s 33ms/step - loss: 0.0131 - val_loss: 0.0041
Epoch 4/20
1/1 [==============================] - 0s 35ms/step - loss: 0.0046 - val_loss: 0.0031
Epoch 5/20
1/1 [==============================] - 0s 32ms/step - loss: 0.0030 - val_loss: 0.0025
Epoch 6/20
1/1 [==============================] - 0s 35ms/step - loss: 0.0026 - val_loss: 0.0025
Epoch 7/20
1/1 [==============================] - 0s 31ms/step - loss: 0.0025 - val_loss: 0.0024
Epoch 8/20
1/1 [==============================] - 0s 32ms/step - loss: 0.0025 - val_loss: 0.0024
Epoch 9/20
1/1 [==============================] - 0s 34ms/step - loss: 0.0025 - val_loss: 0.0024
Epoch 10/20
1/1 [==============================] - 0s 3

In [6]:
#http://localhost:6006

In [7]:
%tensorboard --logdir logs/

A brief overview of the visualizations created in this example and the dashboards (tabs in top navigation bar) where they can be found:

* Scalars show how the loss and metrics change with every epoch. You can use them to also track training speed, learning rate, and other scalar values. Scalars can be found in the Time Series or Scalars dashboards.
* Graphs help you visualize your model. In this case, the Keras graph of layers is shown which can help you ensure it is built correctly. Graphs can be found in the Graphs dashboard.
* Histograms and Distributions show the distribution of a Tensor over time. This can be useful to visualize weights and biases and verify that they are changing in an expected way. Histograms can be found in the Time Series or Histograms dashboards. Distributions can be found in the Distributions dashboard.

Breakdown of the Debugger Interface
The Debugger Dashboard on the Tensorboard consists of five main components:

* __Alerts:__ This top-left section contains a list of alert events detected by the debugger in the debug data from the instrumented TensorFlow program. Each alert indicates a certain anomaly that warrants attention. In our case, this section highlights 499 NaN/∞ events with a salient pink-red color. This confirms our suspicion that the model fails to learn because of the presence of NaNs and/or infinities in its internal tensor values.
* __Python Execution Timeline:__ This is the upper half of the top-middle section. It presents the full history of the eager execution of ops and graphs. Each box of the timeline is marked by the initial letter of the op or graph’s name. We can navigate this timeline by using the navigation buttons and the scrollbar above the timeline.
* __Graph Execution:__ Located at the top-right corner of the GUI, this section will be central to our debugging task. It contains a history of all the floating-type tensors computed inside graphs, i.e., the ones compiled by @tf-functions.
* __Stack Trace:__ The bottom-right section, shows the stack trace of the creation of every single operation on the graph.
* __Source Code:__ The bottom-left section, highlights the source code corresponding to each operation on the graph.
